In [6]:
import json
import csv

In [4]:
file_path = '../data/processed_data/top_building_per_municipality_with_isochrones.geojson'
with open(file_path, 'r') as file:
    geojson_data = json.load(file)

for feature in geojson_data['features']:
    # Initialize an empty set to store unique products
    unique_products = set()

    # Check if 'nearby_farms' property exists
    if 'nearby_farms' in feature['properties']:
        # Iterate over each farm's product list
        for farm in feature['properties']['nearby_farms']:
            # Add products to the set, which handles uniqueness
            unique_products.update(farm['products'])

    # Add the count of unique products as a new property
    feature['properties']['unique_product_count'] = len(unique_products)

output_path = '../data/processed_data/top_building_per_municipality_with_isochrones_and_unique_products.geojson'
with open(output_path, 'w') as outfile:
    json.dump(geojson_data, outfile, indent=4)


In [14]:
# Load the GeoJSON file
file_path = '../data/processed_data/top_building_per_municipality_with_isochrones_and_unique_products.geojson'
with open(file_path, 'r') as file:
    geojson_data = json.load(file)

# Open a new CSV file to write into
output_csv_path = '../data/processed_data/municipality_population_and_products.csv'
with open(output_csv_path, mode='w', newline='') as csv_file:
    fieldnames = ['municipalityName', 'unique_product_count', 'total_pop_5', 'total_pop_10', 'total_pop_15']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Loop through each feature to extract the desired data
    for feature in geojson_data['features']:
        # Extracting data from properties and nested properties
        municipalityName = feature['properties'].get('municipalityName', 'N/A')
        unique_product_count = feature['properties'].get('unique_product_count', 0)

        # Extracting total_pop from nested properties, ensuring proper path and fallback
        total_pop_5 = feature['properties'].get('isochrone_within_5_mode_driving-car', {}).get('features', [{}])[0].get('properties', {}).get('total_pop', 'N/A')
        total_pop_10 = feature['properties'].get('isochrone_within_10_mode_driving-car', {}).get('features', [{}])[0].get('properties', {}).get('total_pop', 'N/A')
        total_pop_15 = feature['properties'].get('isochrone_within_15_mode_driving-car', {}).get('features', [{}])[0].get('properties', {}).get('total_pop', 'N/A')


        # Write the data for the current feature to the CSV
        writer.writerow({
            'municipalityName': municipalityName,
            'unique_product_count': unique_product_count,
            'total_pop_5': total_pop_5,
            'total_pop_10': total_pop_10,
            'total_pop_15': total_pop_15
        })
